In [1]:
import geopandas as gpd
from shapely.geometry import Point
import pandas as pd

In [2]:
# Load real estate data
df = pd.read_csv('../../../data/processed/100k.csv')
df['geometry'] = df.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
gdf_real_estate = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326')


In [ ]:
roads = gpd.read_file(r"D:\CADT\cambodia-latest-free.shp\gis_osm_roads_free_1.shp")

In [ ]:
# Project both to a metric CRS for accurate distance (UTM zone 48N for Cambodia)
gdf_real_estate = gdf_real_estate.to_crs(epsg=32648)
roads = roads.to_crs(epsg=32648)

In [5]:
# Define road types to flag
road_types = [
    'bridleway', 'corridor', 'cycleway', 'disused', 'footway', 'motorway', 'path',
    'pedestrian', 'primary', 'residential', 'road', 'secondary', 'service', 'steps',
    'tertiary', 'track', 'trunk', 'trunk_link', 'unclassified', 'unused'
]

In [ ]:
# Function to flag nearby road types within 100 meters
def find_nearby_road_types(row, roads_gdf, distance=100):
    nearby = roads_gdf[roads_gdf.distance(row.geometry) <= distance]
    flags = {}
    for road_type in road_types:
        flags[f'f_{road_type}'] = int(road_type in nearby['fclass'].values)
    return pd.Series(flags)

In [7]:
# Apply to all real estate points
flags_df = gdf_real_estate.apply(find_nearby_road_types, axis=1, roads_gdf=roads, distance=100)

KeyboardInterrupt: 

In [ ]:
# Combine with original data
result = pd.concat([df.reset_index(drop=True), flags_df.reset_index(drop=True)], axis=1)

# Show result
print(result.head())

  address_subdivision   address_locality       address_line_2  \
0          Phnom Penh          Tuol Kouk  Phsar Depou Ti Muoy   
1          Phnom Penh    Chraoy Chongvar            Bak Kaeng   
2          Phnom Penh  Prampir Meakkakra            Veal Vong   
3          Phnom Penh            Dangkao           Sak Sampov   
4          Phnom Penh        Pur SenChey       Samraong Kraom   

              h_id  price_per_m2  land_area  price   longitude   latitude  \
0  8865846aa3fffff             1        188    188  104.903473  11.580321   
1  886586a6b5fffff             1        164    164  104.941050  11.737009   
2  8865846a85fffff             1        172    172  104.908904  11.561711   
3  88658460c7fffff             1        126    126  104.838857  11.440579   
4  8865846c69fffff             1        136    136  104.803512  11.567503   

   near_Koh_Pich_in_km  ...  f_road  f_secondary  f_service  f_steps  \
0                    5  ...       0            0          0        0   
1 

In [ ]:
result

,address_subdivision,address_locality,address_line_2,h_id,price_per_m2,land_area,price,longitude,latitude,near_Koh_Pich_in_km,...,f_road,f_secondary,f_service,f_steps,f_tertiary,f_track,f_trunk,f_trunk_link,f_unclassified,f_unused
0,Phnom Penh,Tuol Kouk,Phsar Depou Ti Muoy,8865846aa3fffff,1,188,188,104.903473,11.580321,5,...,0,0,0,0,1,0,0,0,0,0
1,Phnom Penh,Chraoy Chongvar,Bak Kaeng,886586a6b5fffff,1,164,164,104.941050,11.737009,20,...,0,0,0,0,0,0,0,0,0,0
2,Phnom Penh,Prampir Meakkakra,Veal Vong,8865846a85fffff,1,172,172,104.908904,11.561711,3,...,0,0,1,0,0,0,0,0,0,0
3,Phnom Penh,Dangkao,Sak Sampov,88658460c7fffff,1,126,126,104.838857,11.440579,16,...,0,0,0,0,0,0,0,0,1,0
4,Phnom Penh,Pur SenChey,Samraong Kraom,8865846c69fffff,1,136,136,104.803512,11.567503,15,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7566,Phnom Penh,Chraoy Chongvar,Bak Kaeng,886586a699fffff,1,182,182,104.929226,11.701893,16,...,0,0,0,0,0,0,0,0,0,0
7567,Phnom Penh,Chraoy Chongvar,Preaek Ta Sek,8865846995fffff,1,212,212,104.899855,11.667508,13,...,0,0,0,0,0,0,0,0,0,0
7568,Phnom Penh,Praek Pnov,Ponsang,8865846d85fffff,1,134,134,104.756877,11.633307,22,...,0,0,0,0,0,0,0,0,0,0
7569,Phnom Penh,Pur SenChey,Kantaok,8865846e31fffff,1,230,230,104.785133,11.523526,17,...,0,0,1,0,0,0,0,0,0,0


In [ ]:
result.to_csv('../../../data/processed/100k.csv', index=False)